In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/movies_metadata.csv
/kaggle/input/ratings.csv
/kaggle/input/links_small.csv
/kaggle/input/links.csv
/kaggle/input/credits.csv
/kaggle/input/ratings_small.csv
/kaggle/input/keywords.csv


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [3]:
df_credits = pd.read_csv('/kaggle/input/credits.csv')
df_keywords = pd.read_csv('/kaggle/input/keywords.csv')
df_links_small = pd.read_csv('/kaggle/input/links_small.csv')
df_links = pd.read_csv('/kaggle/input/links.csv')
df_md = pd.read_csv('/kaggle/input/movies_metadata.csv')
df_ratings = pd.read_csv('/kaggle/input/ratings_small.csv')
# df_ratings=pd.read_csv('/kaggle/input/ratings.csv',header=0)

# Only ratings having value greater than 3
# df_ratings=df_ratings.query('rating>=3')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
df_ratings.shape

(100004, 4)

In [5]:
#only consider ratings from users who have rated over n movies
n=500
df_users = df_ratings.userId.value_counts()
df_users = df_users[df_users>n].index.tolist()

In [6]:
# df_movies=df_ratings.movieId.value_counts()
df_movies=df_ratings.movieId.unique().tolist()
len(df_movies)

9066

In [7]:
len(df_users)

37

In [8]:
df_ratings = df_ratings.query('userId in @df_users')
print(df_ratings.shape)
df_ratings.head(3)

(33545, 4)


,userId,movieId,rating,timestamp
962,15,1,2.0,997938310
963,15,2,2.0,1134521380
964,15,5,4.5,1093070098


In [9]:
df_ratings.groupby('userId')['rating'].count().sort_values(ascending=False)

userId
547    2391
564    1868
624    1735
15     1700
73     1610
452    1340
468    1291
380    1063
311    1019
30     1011
294     947
509     923
580     922
213     910
212     876
472     830
388     792
23      726
457     713
518     707
461     696
232     682
102     678
262     676
475     655
306     645
119     641
654     626
358     617
529     604
575     547
105     525
56      522
353     521
664     519
48      513
587     504
Name: rating, dtype: int64

In [10]:
# Splitting data based on timestamp with 80% as test data and train data

ranks = df_ratings.groupby('userId')['timestamp'].rank(method='first')
counts = df_ratings['userId'].map(df_ratings.groupby('userId')['timestamp'].apply(len))
df_test=df_ratings[(ranks / counts) > 0.8]
print(df_test.shape)
df_train=df_ratings[(ranks / counts) <= 0.8] 
print(df_train.shape)

(6725, 4)
(26820, 4)


In [11]:
ranks = df_ratings.groupby('userId')['timestamp'].rank(method='first')
counts = df_ratings['userId'].map(df_ratings.groupby('userId')['timestamp'].apply(len))
df_test1=df_ratings[(ranks / counts) > 0.8]
print(df_test1.shape)

(6725, 4)


In [12]:
# df_15 = df_test[df_test.userId==380]
# df_15_m = df_15['movieId'].unique().tolist()
# len(df_15_m)
# df_links_15 = df_links.query('movieId in @df_15_m')
# print(df_links_15.shape)
# df_links_15.head(3)

# ## Pre-processing step

# def convert_int(x):
#     try:
#         return int(x)
#     except:
#         return np.nan
    
# df_links_15['tmdbId'] = df_links_15['tmdbId'].apply(convert_int)
# df_links_15[df_links_15['tmdbId'].isnull()]
# df_links_15.drop(8941,inplace=True)
# df_links_15[df_links_15['tmdbId'].isnull()]
# df_links_15['tmdbId']=df_links_15['tmdbId'].astype(int)
# df_links_15 = df_links_15.tmdbId.unique().tolist()
# df_md['id']=df_md['id'].apply(convert_int)
# df_md_15=df_md.query('id in @df_links_15')
# print(df_md_15.shape)

# Popularity recommendation system

In [13]:
df_md.shape

(45466, 24)

In [14]:
df_md['genres'] = df_md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i[
    'name'] for i in x] if isinstance(x, list) else [])
df_md['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

In [15]:
# this is V
vote_counts = df_md[df_md['vote_count'].notnull()]['vote_count'].astype('int')
# print('Vote Counts',vote_counts)

# this is R
vote_averages = df_md[df_md['vote_average'].notnull()]['vote_average'].astype('int')
# print('Vote Averages',vote_averages)

# this is C
C = vote_averages.mean()
print(C)

5.244896612406511


In [16]:
m = vote_counts.quantile(0.95)
m

434.0

In [17]:
# import datetime
# max_year = datetime.date.fromtimestamp(df_train.timestamp.max()).isoformat()
# max_year=max_year.split('-')[0]
# max_year=int(max_year)-1
# max_year

In [18]:
# Pre-processing step for getting year from date by splliting it using '-'
df_md['year'] = pd.to_datetime(df_md['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [19]:
# qualified=df_md[df_md['year']>=str(max_year)]

In [20]:
qualified = df_md[( df_md['vote_count'] >= m) &
                  ( df_md['vote_count'].notnull()) & 
                  ( df_md['vote_average'].notnull())][['title', 
                                                        'year', 
                                                        'vote_count', 
                                                        'vote_average', 
                                                        'popularity', 
                                                        'genres']]

qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [21]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [22]:
m = vote_counts.quantile(0.95)
m

434.0

In [23]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [24]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [25]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


In [26]:
s = df_md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = df_md.drop('genres', axis=1).join(s)
gen_md.head(3).transpose()

,0,0,0
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000,30000000,30000000
homepage,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story
id,862,862,862
imdb_id,tt0114709,tt0114709,tt0114709
original_language,en,en,en
original_title,Toy Story,Toy Story,Toy Story
overview,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
popularity,21.9469,21.9469,21.9469


In [27]:
gen_md.head()

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy


In [28]:
# Generate Chart 
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & 
                   (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity','genre']]
    if qualified.shape[0]!=0:
        qualified['vote_count'] = qualified['vote_count'].astype('int')
        qualified['vote_average'] = qualified['vote_average'].astype('int')  
        qualified['wr'] = qualified.apply(lambda x: 
                            (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C),
                            axis=1)
        qualified = qualified.sort_values('wr', ascending=False).head(30)
   
    return qualified

In [29]:
build_chart('Drama').head(10)

,title,year,vote_count,vote_average,popularity,genre,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,Drama,8.607205
12481,The Dark Knight,2008,12269,8,123.167,Drama,7.983111
22879,Interstellar,2014,11187,8,32.2135,Drama,7.981489
2843,Fight Club,1999,9678,8,63.8696,Drama,7.978628
314,The Shawshank Redemption,1994,8358,8,51.6454,Drama,7.975286
351,Forrest Gump,1994,8147,8,48.3072,Drama,7.974653
834,The Godfather,1972,6024,8,41.1093,Drama,7.965843
24860,The Imitation Game,2014,5895,8,31.5959,Drama,7.965106
359,The Lion King,1994,5520,8,21.6058,Drama,7.962771
18465,The Intouchables,2011,5410,8,16.0869,Drama,7.962025


In [30]:
#format test data
df_test = df_test.copy().groupby('userId')['movieId'].agg({'actual': (lambda x: list(set(x)))})
df_test.head()

,actual
userId,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ..."
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ..."
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515..."
48,"[114180, 109578, 27660, 106002, 34323, 27156, ..."
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081..."


In [31]:
len(df_test.actual[15])

340

In [32]:
user=df_train[df_train['userId']==15]
user.sort_values('timestamp',ascending=False).head(10)

,userId,movieId,rating,timestamp
2307,15,69306,2.0,1389440855
2517,15,106489,2.0,1389440765
1185,15,1147,3.5,1386368101
2479,15,99468,2.5,1386368031
2466,15,97393,3.5,1386368013
2494,15,101895,1.0,1386367999
2497,15,102445,3.0,1386367981
2498,15,102800,2.5,1386367968
2488,15,100714,3.0,1386367955
2469,15,97923,3.0,1386367936


In [33]:
#make recommendations for all members in the test data 
recs = []
recs_temp =[]
for user in df_test.index:
    df_user=df_train[df_train['userId']==user][['movieId','timestamp']].sort_values('timestamp',ascending=False).head(20).set_index('movieId')
    df_genre=gen_md[gen_md.index.isin(df_user.index)]['genre'].unique().tolist()
    if df_genre==[]:
        df_user=df_train[df_train['userId']==user][['movieId','timestamp']].sort_values('timestamp',ascending=False).head(30).set_index('movieId')
        df_genre=gen_md[gen_md.index.isin(df_user.index)]['genre'].unique().tolist()  
    cf_predictions = pd.DataFrame() 
    for genre in list(df_genre):
        if len(df_genre)<6:
            df=build_chart(genre,percentile=0.85).head(30)
        else:
            df=build_chart(genre,percentile=0.85).head(20)
        cf_predictions=cf_predictions.append(df)  
    if cf_predictions.shape[0]!=0:
        recs_temp = cf_predictions.sort_values('wr', ascending=False).index.unique().tolist()
    recs.append(recs_temp)
    
df_test['cf_predictions'] = recs
df_test.head()    

,actual,cf_predictions
userId,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248..."
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1..."
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1..."
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248..."
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8..."


In [34]:
df_test['Length']=df_test['cf_predictions'].apply(lambda x: len(x))

In [35]:
df_test

,actual,cf_predictions,Length
userId,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171
73,"[115713, 514, 122882, 2052, 114180, 4614, 7987...","[10309, 12481, 22879, 2843, 314, 351, 24860, 8...",153
102,"[1028, 1029, 3590, 1019, 1035, 1036, 3606, 360...","[10309, 12481, 22879, 2843, 314, 351, 15480, 8...",182
105,"[10, 11, 25, 36, 5669, 6188, 6711, 51255, 2115...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",198
119,"[2054, 3081, 1034, 4106, 532, 2587, 1564, 2590...","[10309, 12481, 22879, 2843, 314, 351, 15480, 8...",196


# Content based recommendation system: : Using movie description and taglines


In [36]:
df_links_small = df_links_small[df_links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [37]:
## Pre-processing step

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [38]:
df_md['id'] = df_md['id'].apply(convert_int)
df_md[df_md['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[Carousel Productions, Vision View Entertainme...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[Aniplex, GoHands, BROSTA TV, Mardock Scramble...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[Odyssey Media, Pulser Productions, Rogue Stat...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [39]:
df_md = df_md.drop([19730, 29503, 35587])

In [40]:
df_links_small.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [41]:
df_md['id'] = df_md['id'].astype('int')
df_smd = df_md[df_md['id'].isin(df_links_small)]
df_smd.shape

(9099, 25)

In [42]:
df_smd['tagline'] = df_smd['tagline'].fillna('')
df_smd['description'] = df_smd['overview'] + df_smd['tagline']
df_smd['description'] = df_smd['description'].fillna('')


In [43]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_smd['description'])
tfidf_matrix.shape

(9099, 268124)

In [44]:
tfidf_matrix

<9099x268124 sparse matrix of type '<class 'numpy.float64'>'
	with 540591 stored elements in Compressed Sparse Row format>

In [45]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [46]:
df_cos=pd.DataFrame(cosine_sim)
df_cos.head()

,0,1,2,3,4,5,6,7,8,9,...,9089,9090,9091,9092,9093,9094,9095,9096,9097,9098
0,1.000000,0.006805,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.005670,0.000000,0.000000,0.000000,0.000000,0.004780,0.000000,0.003449,0.000000
1,0.006805,1.000000,0.015311,0.0,0.002237,0.014747,0.000000,0.000000,0.033137,0.0,...,0.000000,0.006692,0.000000,0.000000,0.000000,0.001748,0.003671,0.003571,0.007623,0.000000
2,0.000000,0.015311,1.000000,0.0,0.007899,0.000000,0.000000,0.002349,0.000000,0.0,...,0.000000,0.000000,0.000000,0.002311,0.001926,0.002209,0.000000,0.000000,0.002865,0.004722
3,0.000000,0.000000,0.000000,1.0,0.000000,0.002152,0.000000,0.006528,0.000000,0.0,...,0.016816,0.000000,0.003570,0.001992,0.000000,0.000000,0.000000,0.000000,0.008673,0.000000
4,0.000000,0.002237,0.007899,0.0,1.000000,0.000000,0.008263,0.000000,0.011411,0.0,...,0.000000,0.000000,0.006026,0.003362,0.000000,0.003800,0.001869,0.004372,0.001388,0.000000


In [47]:
ratings = pd.read_csv('/kaggle/input/ratings_small.csv')

In [48]:
index_to_movie_id = ratings['movieId']
df_cos.columns = [str(index_to_movie_id[int(col)]) for col in df_cos.columns]
df_cos.index = [index_to_movie_id[idx] for idx in df_cos.index]
df_cos.head()

,31,1029,1061,1129,1172,1263,1287,1293,1339,1343,...,919,1136,1148,1193,1213,1221,1356,1673,1722,1923
31,1.000000,0.006805,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.005670,0.000000,0.000000,0.000000,0.000000,0.004780,0.000000,0.003449,0.000000
1029,0.006805,1.000000,0.015311,0.0,0.002237,0.014747,0.000000,0.000000,0.033137,0.0,...,0.000000,0.006692,0.000000,0.000000,0.000000,0.001748,0.003671,0.003571,0.007623,0.000000
1061,0.000000,0.015311,1.000000,0.0,0.007899,0.000000,0.000000,0.002349,0.000000,0.0,...,0.000000,0.000000,0.000000,0.002311,0.001926,0.002209,0.000000,0.000000,0.002865,0.004722
1129,0.000000,0.000000,0.000000,1.0,0.000000,0.002152,0.000000,0.006528,0.000000,0.0,...,0.016816,0.000000,0.003570,0.001992,0.000000,0.000000,0.000000,0.000000,0.008673,0.000000
1172,0.000000,0.002237,0.007899,0.0,1.000000,0.000000,0.008263,0.000000,0.011411,0.0,...,0.000000,0.000000,0.006026,0.003362,0.000000,0.003800,0.001869,0.004372,0.001388,0.000000


In [49]:
df_cos.iloc[0].sort_values(ascending=False)[:10]

31        1.000000
103228    0.248068
8376      0.230813
1198      0.127272
111759    0.101640
595       0.080479
34326     0.072698
1         0.071859
260       0.059213
920       0.056647
Name: 31, dtype: float64

In [50]:
df_cos[df_cos.index==31].head(1)

,31,1029,1061,1129,1172,1263,1287,1293,1339,1343,...,919,1136,1148,1193,1213,1221,1356,1673,1722,1923
31,1.0,0.006805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00567,0.0,0.0,0.0,0.0,0.00478,0.0,0.003449,0.0


In [51]:
    ss1 = list(enumerate(cosine_sim[629]))
#     ss1 = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     ss1 = sim_scores[1:31]

In [52]:
    ss = list(enumerate(df_cos[df_cos.index==31].head(1)))
#     ss = sorted(ss, key=lambda x: x[1], reverse=True)
#     ss = ss[1:31]

In [53]:
df_smd.shape

(9099, 26)

In [54]:
df_smd = df_smd.reset_index()

In [55]:
titles = df_smd['title']
indices = pd.Series(df_smd.index, index=df_smd['title'])

In [56]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [57]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [58]:
def get_recommendations1(idx):
#     idx = indices[title]
    movie_indices=[]
    if idx < 9099:
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]
        movie_indices = [i[0] for i in sim_scores]
    return movie_indices

In [59]:
get_recommendations1(692)

[973, 8387, 3509, 4196, 29]

In [60]:
#make recommendations for all members in the test data 
# Get the recent 20 movies rated by the user and find 5 most similar movies t
recs = []
imx=index_to_movie_id.tolist()
for user in df_test.index:
    df_user=df_train[df_train['userId']==user][['movieId','timestamp']].sort_values('timestamp',ascending=False).head(30).set_index('movieId')
    df_user=df_user.index.tolist()
    cf_predictions = []
    for movieId in list(df_user):
        if len(cf_predictions)<150:
            cf_predictions = cf_predictions + get_recommendations1(imx.index(movieId))
    recs.append(cf_predictions)

df_test['cf_predictions_td'] = recs
df_test.head()

,actual,cf_predictions,Length,cf_predictions_td
userId,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,..."
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187..."
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850..."
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381..."
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671..."


In [61]:
df_test['L_td']=df_test['cf_predictions_td'].apply(lambda x: len(x)) 

In [62]:
df_test['l_a']=df_test['actual'].apply(lambda x: len(x))

In [63]:
df_test.head(10)

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a
userId,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105
73,"[115713, 514, 122882, 2052, 114180, 4614, 7987...","[10309, 12481, 22879, 2843, 314, 351, 24860, 8...",153,"[5808, 8879, 5197, 3601, 3465, 7094, 1900, 638...",140,322
102,"[1028, 1029, 3590, 1019, 1035, 1036, 3606, 360...","[10309, 12481, 22879, 2843, 314, 351, 15480, 8...",182,"[6735, 2792, 6172, 8891, 7114, 435, 2785, 7405...",130,136
105,"[10, 11, 25, 36, 5669, 6188, 6711, 51255, 2115...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",198,"[3453, 6226, 3542, 5349, 3658, 6244, 8769, 891...",105,105
119,"[2054, 3081, 1034, 4106, 532, 2587, 1564, 2590...","[10309, 12481, 22879, 2843, 314, 351, 15480, 8...",196,"[8206, 2678, 8951, 6971, 5018, 7640, 817, 1653...",105,129


# Content based RS : Using movie description, taglines, keywords, cast, director and genres

In [64]:
df_credits = pd.read_csv('/kaggle/input/credits.csv')

In [65]:
df_keywords['id'] = df_keywords['id'].astype('int')
df_credits['id'] = df_credits['id'].astype('int')
df_md['id'] = df_md['id'].astype('int')

In [66]:
df_md = df_md.merge(df_credits, on='id')
df_md = df_md.merge(df_keywords, on='id')

In [67]:
df_smd = df_md[df_md['id'].isin(df_links_small)]
df_smd.shape

(9219, 28)

In [68]:
df_smd['cast'] = df_smd['cast'].apply(literal_eval)
df_smd['crew'] = df_smd['crew'].apply(literal_eval)
df_smd['keywords'] = df_smd['keywords'].apply(literal_eval)
df_smd['cast_size'] = df_smd['cast'].apply(lambda x: len(x))
df_smd['crew_size'] = df_smd['crew'].apply(lambda x: len(x))

In [69]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [70]:
df_smd['director'] = df_smd['crew'].apply(get_director)
df_smd['cast'] = df_smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df_smd['cast'] = df_smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
df_smd['keywords'] = df_smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [71]:
df_smd['cast'] = df_smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
df_smd['director'] = df_smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
df_smd['director'] = df_smd['director'].apply(lambda x: [x,x, x])

In [72]:
s = df_smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [73]:
s = s[s > 1]

In [74]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [75]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [76]:
df_smd['keywords'] = df_smd['keywords'].apply(filter_keywords)
df_smd['keywords'] = df_smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
df_smd['keywords'] = df_smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [77]:
df_smd['soup'] = df_smd['keywords'] + df_smd['cast'] + df_smd['director'] + df_smd['genres']
df_smd['soup'] = df_smd['soup'].apply(lambda x: ' '.join(x))

In [78]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df_smd['soup'])

In [79]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [80]:
df_smd = df_smd.reset_index()
titles = df_smd['title']
indices = pd.Series(df_smd.index, index=df_smd['title'])

In [81]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [82]:
get_recommendations('Inception').head(10)

6623                             The Prestige
3381                                  Memento
4145                                 Insomnia
2085                                Following
8031                    The Dark Knight Rises
8613                             Interstellar
6981                          The Dark Knight
6218                            Batman Begins
5638    Sky Captain and the World of Tomorrow
8500                                  Don Jon
Name: title, dtype: object

In [83]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:50]
    movie_indices = [i[0] for i in sim_scores]
    
    df_movies = df_smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = df_movies[df_movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df_movies[df_movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.80)
    qualified = df_movies[(df_movies['vote_count'] >= m) & (df_movies['vote_count'].notnull()) & 
                       (df_movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
#     qualified = qualified[qualified['year']>=str(max_year-5)].head(10)    
    return qualified

In [84]:
# qualification = 60 %
# max year = 2014
# similar movies = 100
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
7583,Kick-Ass,4747,7,2010,6.852979
524,Batman,2145,7,1989,6.704647
8467,Kick-Ass 2,2275,6,2013,5.879027
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943


In [85]:
# qualification = 60 %
# max year = 2014
# similar movies = 50
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
7583,Kick-Ass,4747,7,2010,6.852979
524,Batman,2145,7,1989,6.704647
8467,Kick-Ass 2,2275,6,2013,5.879027
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943


In [86]:
# with no year restriction, 25 similar movies and qualification range 60%
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
7583,Kick-Ass,4747,7,2010,6.852979
524,Batman,2145,7,1989,6.704647
8467,Kick-Ass 2,2275,6,2013,5.879027
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943


In [87]:
# 50 similar movies, max year -5, quantile range 80
improved_recommendations('Inception')

,title,vote_count,vote_average,year,wr
6981,The Dark Knight,12269,8,2008,7.905871
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
7841,Source Code,2752,7,2011,6.760918
8207,Looper,4777,6,2012,5.937111
7286,X-Men Origins: Wolverine,4086,6,2009,5.927497
7991,In Time,3512,6,2011,5.916950


In [88]:
def improved_recommendations1(idx):
#     idx = indices[title]
    movie_indices=[]
    if idx < 9219:
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:50]
        movie_indices = [i[0] for i in sim_scores]

        df_movies = df_smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
        vote_counts = df_movies[df_movies['vote_count'].notnull()]['vote_count'].astype('int')
        vote_averages = df_movies[df_movies['vote_average'].notnull()]['vote_average'].astype('int')
        C = vote_averages.mean()
        m = vote_counts.quantile(0.80)
        qualified = df_movies[(df_movies['vote_count'] >= m) & (df_movies['vote_count'].notnull()) & 
                           (df_movies['vote_average'].notnull())]
        qualified['vote_count'] = qualified['vote_count'].astype('int')
        qualified['vote_average'] = qualified['vote_average'].astype('int')
        qualified['wr'] = qualified.apply(weighted_rating, axis=1)
        qualified = qualified.sort_values('wr', ascending=False).head(5)
#         qualified = qualified[qualified['year']>=str(max_year-5)].head(10)    
        return qualified.index.tolist()
    else:
        return movie_indices

In [89]:
#make recommendations for all members in the test data
# Get the recent 20 movies rated by the user and find 5 most similar movies t
recs = []
imx=index_to_movie_id.tolist()
for user in df_test.index:
    df_user=df_train[df_train['userId']==user][['movieId','timestamp']].sort_values('timestamp',ascending=False).head(30).set_index('movieId')
    df_user=df_user.index.tolist()
    cf_predictions = []
    for movieId in list(df_user):
        cf_predictions = cf_predictions + improved_recommendations1(imx.index(movieId))
    recs.append(cf_predictions)

df_test['cf_predictions_kcdg'] = recs
df_test.head()

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a,cf_predictions_kcdg
userId,,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340,"[523, 3296, 988, 4119, 5538, 100, 6592, 8394, ..."
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146,"[5133, 1539, 4977, 2363, 5075, 1584, 4146, 604..."
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203,"[535, 6881, 6948, 2557, 1853, 8666, 8226, 1463..."
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103,"[1056, 4619, 6476, 7632, 9100, 4951, 4960, 886..."
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105,"[1077, 8180, 7061, 3235, 8612, 8456, 8435, 766..."


In [90]:
df_test['l_kcdg']=df_test['cf_predictions_kcdg'].apply(lambda x: len(x))
df_test.head()

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a,cf_predictions_kcdg,l_kcdg
userId,,,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340,"[523, 3296, 988, 4119, 5538, 100, 6592, 8394, ...",150
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146,"[5133, 1539, 4977, 2363, 5075, 1584, 4146, 604...",150
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203,"[535, 6881, 6948, 2557, 1853, 8666, 8226, 1463...",150
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103,"[1056, 4619, 6476, 7632, 9100, 4951, 4960, 886...",150
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105,"[1077, 8180, 7061, 3235, 8612, 8456, 8435, 766...",150


In [91]:
df_test

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a,cf_predictions_kcdg,l_kcdg
userId,,,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340,"[523, 3296, 988, 4119, 5538, 100, 6592, 8394, ...",150
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146,"[5133, 1539, 4977, 2363, 5075, 1584, 4146, 604...",150
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203,"[535, 6881, 6948, 2557, 1853, 8666, 8226, 1463...",150
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103,"[1056, 4619, 6476, 7632, 9100, 4951, 4960, 886...",150
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105,"[1077, 8180, 7061, 3235, 8612, 8456, 8435, 766...",150
73,"[115713, 514, 122882, 2052, 114180, 4614, 7987...","[10309, 12481, 22879, 2843, 314, 351, 24860, 8...",153,"[5808, 8879, 5197, 3601, 3465, 7094, 1900, 638...",140,322,"[731, 8975, 6516, 7086, 7134, 8871, 7672, 1263...",140
102,"[1028, 1029, 3590, 1019, 1035, 1036, 3606, 360...","[10309, 12481, 22879, 2843, 314, 351, 15480, 8...",182,"[6735, 2792, 6172, 8891, 7114, 435, 2785, 7405...",130,136,"[298, 7962, 7113, 3428, 1762, 1058, 748, 1348,...",130
105,"[10, 11, 25, 36, 5669, 6188, 6711, 51255, 2115...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",198,"[3453, 6226, 3542, 5349, 3658, 6244, 8769, 891...",105,105,"[8419, 1146, 8934, 1500, 3898, 9043, 1305, 165...",105
119,"[2054, 3081, 1034, 4106, 532, 2587, 1564, 2590...","[10309, 12481, 22879, 2843, 314, 351, 15480, 8...",196,"[8206, 2678, 8951, 6971, 5018, 7640, 817, 1653...",105,129,"[8522, 8840, 1058, 1306, 5987, 2425, 7045, 854...",105


In [92]:
def improved_recommendations2(idx):
#     idx = indices[title]
    movie_indices=[]
    if idx < 9219:
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]
        movie_indices = [i[0] for i in sim_scores]

#         df_movies = df_smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
#         vote_counts = df_movies[df_movies['vote_count'].notnull()]['vote_count'].astype('int')
#         vote_averages = df_movies[df_movies['vote_average'].notnull()]['vote_average'].astype('int')
#         C = vote_averages.mean()
#         m = vote_counts.quantile(0.80)
#         qualified = df_movies[(df_movies['vote_count'] >= m) & (df_movies['vote_count'].notnull()) & 
#                            (df_movies['vote_average'].notnull())]
#         qualified['vote_count'] = qualified['vote_count'].astype('int')
#         qualified['vote_average'] = qualified['vote_average'].astype('int')
#         qualified['wr'] = qualified.apply(weighted_rating, axis=1)
#         qualified = qualified.sort_values('wr', ascending=False).head(10)
# #         qualified = qualified[qualified['year']>=str(max_year-5)].head(10)    
#         return qualified.index.tolist()
    return movie_indices

In [93]:
#make recommendations for all members in the test data 
# Get the recent 20 movies rated by the user and find 5 most similar movies t
recs = []
imx=index_to_movie_id.tolist()
for user in df_test.index:
    df_user=df_train[df_train['userId']==user][['movieId','timestamp']].sort_values('timestamp',ascending=False).head(40).set_index('movieId')
    df_user=df_user.index.tolist()
    cf_predictions = []
    for movieId in list(df_user):
        cf_predictions = cf_predictions + improved_recommendations2(imx.index(movieId))
    recs.append(cf_predictions)

df_test['cf_predictions_kcdg2'] = recs
df_test.head()

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a,cf_predictions_kcdg,l_kcdg,cf_predictions_kcdg2
userId,,,,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340,"[523, 3296, 988, 4119, 5538, 100, 6592, 8394, ...",150,"[1918, 3237, 271, 7306, 8370, 919, 4827, 3427,..."
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146,"[5133, 1539, 4977, 2363, 5075, 1584, 4146, 604...",150,"[2122, 1822, 329, 2363, 4977, 1192, 2207, 7228..."
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203,"[535, 6881, 6948, 2557, 1853, 8666, 8226, 1463...",150,"[2899, 5167, 8379, 3245, 2106, 3433, 1463, 121..."
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103,"[1056, 4619, 6476, 7632, 9100, 4951, 4960, 886...",150,"[2058, 7421, 5496, 8811, 9100, 5255, 7650, 239..."
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105,"[1077, 8180, 7061, 3235, 8612, 8456, 8435, 766...",150,"[8145, 8474, 306, 7061, 1789, 4796, 2978, 5093..."


In [94]:
# df_test['l_kcdg2']=df_test['cf_predictions_kcdg2'].apply(lambda x: len(x)) ABDC
# df_test.head()

# Evaluation

In [95]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
from sklearn.metrics import mean_squared_error
from math import sqrt
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [96]:
def _ark(actual, predicted, k=10):
    """
    Computes the average recall at k.
    Parameters
    ----------
    actual : list
        A list of actual items to be predicted
    predicted : list
        An ordered list of predicted items
    k : int, default = 10
        Number of predictions to consider
    Returns:
    -------
    score : int
        The average recall at k.
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return (score / len(actual)*100)

In [97]:
def mark(actual, predicted, k=10):
    """
    Computes the mean average recall at k.
    Parameters
    ----------
    actual : a list of lists
        Actual items to be predicted
        example: [['A', 'B', 'X'], ['A', 'B', 'Y']]
    predicted : a list of lists
        Ordered predictions
        example: [['X', 'Y', 'Z'], ['X', 'Y', 'Z']]
    Returns:
    -------
        mark: int
            The mean average recall at k (mar@k)
    """
    return np.mean([_ark(a,p,k) for a,p in zip(actual, predicted)])

In [98]:
actual = df_test.actual.values.tolist() 
cf_predictions_pop = df_test.cf_predictions.values.tolist()
cf_predictions_td = df_test.cf_predictions_td.values.tolist()
cf_predictions_kcdg = df_test.cf_predictions_kcdg.values.tolist()
cf_predictions_kcdg2 = df_test.cf_predictions_kcdg2.values.tolist()

In [99]:
# pop_mark = [] ABDC
# for K in np.arange(1, 150):
#     pop_mark.extend([mark(actual, cf_predictions_kcdg2, k=K)])
# pop_mark

# Content Based Recommender using User Profile and Movie 

In [100]:
len(df_test1['movieId'].unique())

4245

In [101]:
df_user=df_test1[df_test1.userId==547]
df_user.shape

(479, 4)

In [102]:
df_test1_links = df_links.merge(df_user,on='movieId')
print(df_test1_links.shape)
df_test1_links.head()

(478, 6)


,movieId,imdbId,tmdbId,userId,rating,timestamp
0,11,112346,9087.0,547,3.0,1418149949
1,125,116324,2074.0,547,5.0,1370674374
2,380,111503,36955.0,547,3.0,1313414054
3,1088,92890,88.0,547,5.0,1384462255
4,1172,95765,11216.0,547,5.0,1373125067


In [103]:
df_test1_links.drop(df_test1_links[df_test1_links['tmdbId'].isnull()].index,inplace=True)

In [104]:
df_test1_links['id']=df_test1_links['tmdbId'].astype(int)
df_test1_links.drop('tmdbId',axis=1,inplace=True)
df_test1_links.head()

,movieId,imdbId,userId,rating,timestamp,id
0,11,112346,547,3.0,1418149949,9087
1,125,116324,547,5.0,1370674374,2074
2,380,111503,547,3.0,1313414054,36955
3,1088,92890,547,5.0,1384462255,88
4,1172,95765,547,5.0,1373125067,11216


In [105]:
df_user_data_with_tags = df_md.reset_index().merge(df_test1_links, on='id')
df_user_data_with_tags.shape

(486, 34)

In [106]:
df_user_data_with_tags.head(3).transpose()

,0,1,2
index,10,123,375
adult,False,False,False
belongs_to_collection,NaN,NaN,NaN
budget,62000000,7000000,115000000
genres,"[Comedy, Drama, Romance]","[Comedy, Romance]","[Action, Thriller]"
homepage,NaN,NaN,NaN
id,9087,2074,36955
imdb_id,tt0112346,tt0116324,tt0111503
original_language,en,en,en
original_title,The American President,Flirting with Disaster,True Lies


In [107]:
df_user_data_with_tags['index'].values[-10:]

array([36768, 36769, 37326, 37327, 39032, 39345, 40528, 40554, 40688,
       41669])

In [108]:
df_smd.tail()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,soup
9214,40952,False,NaN,8000000,[Drama],NaN,159550,tt0255313,en,The Last Brickmaker in America,...,7.0,1.0,2001,"[sidneypoitier, wendycrewson, jayo.sanders]","[{'credit_id': '544475aac3a36819fb000578', 'de...",[friendship],7,2,"[greggchampion, greggchampion, greggchampion]",friendship sidneypoitier wendycrewson jayo.san...
9215,41172,False,NaN,1000000,"[Thriller, Romance]",NaN,392572,tt5165344,hi,रुस्तम,...,7.3,25.0,2016,"[akshaykumar, ileanad'cruz, eshagupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],14,16,"[tinusureshdesai, tinusureshdesai, tinusureshd...",bollywood akshaykumar ileanad'cruz eshagupta t...
9216,41225,False,NaN,15050000,"[Adventure, Drama, History, Romance]",NaN,402672,tt3859980,hi,Mohenjo Daro,...,6.7,26.0,2016,"[hrithikroshan, poojahegde, kabirbedi]","[{'credit_id': '57cd5d3592514179d50018e8', 'de...",[bollywood],12,16,"[ashutoshgowariker, ashutoshgowariker, ashutos...",bollywood hrithikroshan poojahegde kabirbedi a...
9217,41391,False,NaN,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",NaN,315011,tt4262980,ja,シン・ゴジラ,...,6.6,152.0,2016,"[hirokihasegawa, yutakatakenouchi, satomiishih...","[{'credit_id': '560892fa92514177550018b2', 'de...","[monster, godzilla, giantmonst, destruct, kaiju]",49,27,"[hideakianno, hideakianno, hideakianno]",monster godzilla giantmonst destruct kaiju hir...
9218,41669,False,NaN,0,"[Documentary, Music]",http://www.thebeatlesliveproject.com/,391698,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,...,7.6,92.0,2016,"[paulmccartney, ringostarr, johnlennon]","[{'credit_id': '57057c6cc3a3680dca000285', 'de...","[music, documentari]",28,7,"[ronhoward, ronhoward, ronhoward]",music documentari paulmccartney ringostarr joh...


In [109]:
df_smd['soup'].head()

0    jealousi toy boy friendship friend rivalri boy...
1    boardgam disappear basedonchildren'sbook newho...
2    fish bestfriend duringcreditssting waltermatth...
3    basedonnovel interracialrelationship singlemot...
4    babi midlifecrisi confid age daughter motherda...
Name: soup, dtype: object

In [110]:
tf_soup = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix_soup = tf.fit_transform(df_smd['soup'])

In [111]:
tfidf_matrix_soup.shape

(9219, 107377)

In [112]:
idx2movie = pd.Series(df_smd.index, index=df_smd['index'])
idx2movie[-10:]

index
40787    9209
40831    9210
40837    9211
40920    9212
40939    9213
40952    9214
41172    9215
41225    9216
41391    9217
41669    9218
dtype: int64

In [113]:
idx2movie[df_user_data_with_tags['index'].values[-10:]].values

array([9150, 9151, 9160, 9161, 9177, 9182, 9204, 9205, 9207, 9218])

In [114]:
user_profile = np.dot(tfidf_matrix_soup[idx2movie[df_user_data_with_tags['index'].values].values].toarray().T, df_user_data_with_tags['rating'].values)

In [115]:
C = cosine_similarity(np.atleast_2d(user_profile), tfidf_matrix_soup)

In [116]:
R = np.argsort(C)[:, ::-1]

In [117]:
recommendations = [i for i in R[0] if i not in df_user_data_with_tags['index'].values]

In [118]:
df_smd['title'][recommendations].head(20)

8753                     Force Majeure
8748                     Force Majeure
8754                     Force Majeure
8752                     Force Majeure
8751                     Force Majeure
8749                     Force Majeure
8750                     Force Majeure
8755                     Force Majeure
8255           Silver Linings Playbook
9073                               Joy
8549                   American Hustle
7747                       The Fighter
6145    A League of Ordinary Gentlemen
7613               Life After Tomorrow
7866              Voices from the List
9070                         Boy Crazy
8294              Little Miss Broadway
4143                    The Last Waltz
113             Flirting with Disaster
4072                     Boxcar Bertha
Name: title, dtype: object

In [119]:
df_smd['title'].head()

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

In [120]:
df_smd['title'][recommendations].head(20)

8753                     Force Majeure
8748                     Force Majeure
8754                     Force Majeure
8752                     Force Majeure
8751                     Force Majeure
8749                     Force Majeure
8750                     Force Majeure
8755                     Force Majeure
8255           Silver Linings Playbook
9073                               Joy
8549                   American Hustle
7747                       The Fighter
6145    A League of Ordinary Gentlemen
7613               Life After Tomorrow
7866              Voices from the List
9070                         Boy Crazy
8294              Little Miss Broadway
4143                    The Last Waltz
113             Flirting with Disaster
4072                     Boxcar Bertha
Name: title, dtype: object

In [121]:
df_smd['index'][recommendations].head(50).values

array([24364, 24359, 24365, 24363, 24362, 24360, 24361, 24366, 19920,
       32669, 22363, 16321,  9918, 15434, 17202, 32604, 20171,  5299,
         123,  5189, 14997, 10516, 17763, 19882, 20141, 22329, 20664,
       17448, 35325, 17501, 15752,   568, 41669, 20246, 10637, 20613,
       30220, 26077,  8857,  5082, 17940,  8266, 14329,  9623, 10954,
        8194, 21198, 13408, 12330, 18282])

In [122]:
#make recommendations for all members in the test data
# Get the recent 20 movies rated by the user and find 5 most similar movies t
recs = []
for user in df_test.index:
    df_user=df_train[df_train['userId']==user]
    df_user_links = df_links.merge(df_user,on='movieId')
    df_user_links.drop(df_user_links[df_user_links['tmdbId'].isnull()].index,inplace=True)
    df_user_links['id']=df_user_links['tmdbId'].astype(int)
    df_user_links.drop('tmdbId',axis=1,inplace=True)
    df_user_data_with_tags = df_md.reset_index().merge(df_user_links, on='id')
    user_profile = np.dot(tfidf_matrix_soup[idx2movie[df_user_data_with_tags['index'].values].values].toarray().T, df_user_data_with_tags['rating'].values)
    C = cosine_similarity(np.atleast_2d(user_profile), tfidf_matrix_soup)
    R = np.argsort(C)[:, ::-1]
    recommendations = [i for i in R[0] if i not in df_user_data_with_tags['index'].values]
    recs.append(df_smd['index'][recommendations].head(150).values.tolist())

df_test['cf_predictions_up'] = recs
df_test.head()

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a,cf_predictions_kcdg,l_kcdg,cf_predictions_kcdg2,cf_predictions_up
userId,,,,,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340,"[523, 3296, 988, 4119, 5538, 100, 6592, 8394, ...",150,"[1918, 3237, 271, 7306, 8370, 919, 4827, 3427,...","[5359, 5932, 5927, 5934, 5928, 5929, 5930, 593..."
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146,"[5133, 1539, 4977, 2363, 5075, 1584, 4146, 604...",150,"[2122, 1822, 329, 2363, 4977, 1192, 2207, 7228...","[7416, 7414, 7419, 7415, 7420, 7418, 7421, 741..."
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203,"[535, 6881, 6948, 2557, 1853, 8666, 8226, 1463...",150,"[2899, 5167, 8379, 3245, 2106, 3433, 1463, 121...","[5862, 4505, 16076, 8911, 1220, 2321, 1614, 32..."
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103,"[1056, 4619, 6476, 7632, 9100, 4951, 4960, 886...",150,"[2058, 7421, 5496, 8811, 9100, 5255, 7650, 239...","[5529, 6281, 2911, 7016, 9786, 25543, 5895, 13..."
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105,"[1077, 8180, 7061, 3235, 8612, 8456, 8435, 766...",150,"[8145, 8474, 306, 7061, 1789, 4796, 2978, 5093...","[5933, 5930, 5928, 5932, 5929, 5934, 5931, 592..."


In [123]:
df_test['l_up']=df_test['cf_predictions_up'].apply(lambda x: len(x))
df_test.head()

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a,cf_predictions_kcdg,l_kcdg,cf_predictions_kcdg2,cf_predictions_up,l_up
userId,,,,,,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340,"[523, 3296, 988, 4119, 5538, 100, 6592, 8394, ...",150,"[1918, 3237, 271, 7306, 8370, 919, 4827, 3427,...","[5359, 5932, 5927, 5934, 5928, 5929, 5930, 593...",150
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146,"[5133, 1539, 4977, 2363, 5075, 1584, 4146, 604...",150,"[2122, 1822, 329, 2363, 4977, 1192, 2207, 7228...","[7416, 7414, 7419, 7415, 7420, 7418, 7421, 741...",150
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203,"[535, 6881, 6948, 2557, 1853, 8666, 8226, 1463...",150,"[2899, 5167, 8379, 3245, 2106, 3433, 1463, 121...","[5862, 4505, 16076, 8911, 1220, 2321, 1614, 32...",150
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103,"[1056, 4619, 6476, 7632, 9100, 4951, 4960, 886...",150,"[2058, 7421, 5496, 8811, 9100, 5255, 7650, 239...","[5529, 6281, 2911, 7016, 9786, 25543, 5895, 13...",150
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105,"[1077, 8180, 7061, 3235, 8612, 8456, 8435, 766...",150,"[8145, 8474, 306, 7061, 1789, 4796, 2978, 5093...","[5933, 5930, 5928, 5932, 5929, 5934, 5931, 592...",150


Final results

In [124]:
# Recommender System based on Tags and description 
cf_predictions = df_test.cf_predictions.values.tolist()
print('MAR@K25:',mark(actual, cf_predictions, k=25))
print('MAR@K50:',mark(actual, cf_predictions, k=50))
print('MAR@K75:',mark(actual, cf_predictions, k=75))
print('MAR@K100:',mark(actual, cf_predictions, k=100))
print('MAR@K125:',mark(actual, cf_predictions, k=125))
print('MAR@K150:',mark(actual, cf_predictions, k=150))

MAR@K25: 0.015907605756150165
MAR@K50: 0.022666389283799087
MAR@K75: 0.023872775686770372
MAR@K100: 0.025233953330343638
MAR@K125: 0.026239457861320776
MAR@K150: 0.026441597100893176


In [125]:
# Recommender System based on Tags and description
cf_predictions_td = df_test.cf_predictions_td.values.tolist()
print('MAR@K25:',mark(actual, cf_predictions_td, k=25))
print('MAR@K50:',mark(actual, cf_predictions_td, k=50))
print('MAR@K75:',mark(actual, cf_predictions_td, k=75))
print('MAR@K100:',mark(actual, cf_predictions_td, k=100))
print('MAR@K125:',mark(actual, cf_predictions_td, k=125))
print('MAR@K150:',mark(actual, cf_predictions_td, k=150))

MAR@K25: 0.03192144870237758
MAR@K50: 0.03711304899974399
MAR@K75: 0.044546079410829956
MAR@K100: 0.04911629973788136
MAR@K125: 0.05114257095382057
MAR@K150: 0.05154793582273909


In [126]:
# Recommender System based Keywords,Description,Tags, and Genres 
cf_predictions_kcdg = df_test.cf_predictions_kcdg.values.tolist()
print('MAR@K25:',mark(actual, cf_predictions_kcdg, k=25))
print('MAR@K50:',mark(actual, cf_predictions_kcdg, k=50))
print('MAR@K75:',mark(actual, cf_predictions_kcdg, k=75))
print('MAR@K100:',mark(actual, cf_predictions_kcdg, k=100))
print('MAR@K125:',mark(actual, cf_predictions_kcdg, k=125))
print('MAR@K150:',mark(actual, cf_predictions_kcdg, k=150))

MAR@K25: 0.03827603184551847
MAR@K50: 0.04418078607616909
MAR@K75: 0.05038863517358141
MAR@K100: 0.054653091096447386
MAR@K125: 0.05658903539893408
MAR@K150: 0.05665018251890473


In [127]:
# Recommender System based Keywords,description,Tags and Genres -2 
cf_predictions_kcdg2 = df_test.cf_predictions_kcdg2.values.tolist()
print('MAR@K25:',mark(actual, cf_predictions_kcdg2, k=25))
print('MAR@K50:',mark(actual, cf_predictions_kcdg2, k=50))
print('MAR@K75:',mark(actual, cf_predictions_kcdg2, k=75))
print('MAR@K100:',mark(actual, cf_predictions_kcdg2, k=100))
print('MAR@K125:',mark(actual, cf_predictions_kcdg2, k=125))
print('MAR@K150:',mark(actual, cf_predictions_kcdg2, k=150))

MAR@K25: 0.07421241458906006
MAR@K50: 0.08433086047526768
MAR@K75: 0.0921465973639233
MAR@K100: 0.09322627229444212
MAR@K125: 0.09634785870098049
MAR@K150: 0.09967052347022941


In [128]:
# At K=25
actual = df_test.actual.values.tolist()
cf_predictions_up = df_test.cf_predictions_up.values.tolist()
print('MAR@K25:',mark(actual, cf_predictions_up, k=25))
print('MAR@K50:',mark(actual, cf_predictions_up, k=50))
print('MAR@K75:',mark(actual, cf_predictions_up, k=75))
print('MAR@K100:',mark(actual, cf_predictions_up, k=100))
print('MAR@K125:',mark(actual, cf_predictions_up, k=125))
print('MAR@K150:',mark(actual, cf_predictions_up, k=150))

MAR@K25: 0.037683861661614415
MAR@K50: 0.054260339601174296
MAR@K75: 0.06168902903189486
MAR@K100: 0.06912426974182918
MAR@K125: 0.0749751160859991
MAR@K150: 0.08370227207010168


In [129]:
def recommender_precision(predicted, actual):
    """
    Computes the precision of each user's list of recommendations, and averages precision over all users.
    ----------
    actual : a list of lists
        Actual items to be predicted
        example: [['A', 'B', 'X'], ['A', 'B', 'Y']]
    predicted : a list of lists
        Ordered predictions
        example: [['X', 'Y', 'Z'], ['X', 'Y', 'Z']]
    Returns:
    -------
        precision: int
    """
    def calc_precision(predicted, actual):
        prec = [value for value in predicted if value in actual]
        prec = np.round(float(len(prec)) / float(len(predicted))*100, 4)
        return prec

    precision = np.mean(list(map(calc_precision,predicted,actual)))
    return precision

In [130]:
print('Precision for popularity based recommender system is:',recommender_precision(actual,cf_predictions))
print('Precision for RS based on Tags & Description is:',recommender_precision(actual,cf_predictions_td))
print('Precision for RS based on Keywords,Credits,Description,Genres is:',recommender_precision(actual,cf_predictions_kcdg))
print('Precision for RS based on Keywords,Credits,Description,Genres 2 is:',recommender_precision(actual,cf_predictions_kcdg2))
print('Precision for RS based on User Profile is:',recommender_precision(actual,cf_predictions_up))

Precision for popularity based recommender system is: 0.6709540540540542
Precision for RS based on Tags & Description is: 0.9507810810810811
Precision for RS based on Keywords,Credits,Description,Genres is: 0.8858945945945946
Precision for RS based on Keywords,Credits,Description,Genres 2 is: 1.1358675675675678
Precision for RS based on User Profile is: 1.4439297297297295


# Hybrid Model using User Profile and NCF

In [131]:
df_test1.head()

,userId,movieId,rating,timestamp
1059,15,371,2.0,1443385370
1060,15,372,3.0,1465793100
1065,15,429,1.0,1465955324
1085,15,520,3.5,1443384915
1150,15,913,4.0,1416119525


In [132]:
from sklearn.metrics import mean_squared_error
# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model
from keras.layers import merge

Using TensorFlow backend.


In [133]:
# Create user- & movie-id mapping
user_id_mapping = {id:i for i, id in enumerate(df_ratings['userId'].unique())}
movie_id_mapping = {id:i for i, id in enumerate(df_ratings['movieId'].unique())}


# Create correctly mapped train- & testset
train_user_data = df_train['userId'].map(user_id_mapping)
train_movie_data = df_train['movieId'].map(movie_id_mapping)

test_user_data = df_test1['userId'].map(user_id_mapping)
test_movie_data = df_test1['movieId'].map(movie_id_mapping)


# Get input variable-sizes
users = len(user_id_mapping)
movies = len(movie_id_mapping)
embedding_size = 10


##### Create model
# Set input layers
user_id_input = Input(shape=[1], name='User')
movie_id_input = Input(shape=[1], name='Movie')

# Create embedding layers for users and movies
user_embedding = Embedding(output_dim=embedding_size, 
                           input_dim=users,
                           input_length=1, 
                           name='user_embedding')(user_id_input)
movie_embedding = Embedding(output_dim=embedding_size, 
                            input_dim=movies,
                            input_length=1, 
                            name='item_embedding')(movie_id_input)

# Reshape the embedding layers
user_vector = Reshape([embedding_size])(user_embedding)
movie_vector = Reshape([embedding_size])(movie_embedding)

# Compute dot-product of reshaped embedding layers as prediction
y = Dot(1, normalize=False)([user_vector, movie_vector])

# Setup model
model = Model(inputs=[user_id_input, movie_id_input], outputs=y)
model.compile(loss='mse', optimizer='adam')


# Fit model
model.fit([train_user_data, train_movie_data],
          df_train['rating'],
          batch_size=256, 
          epochs=20,
          validation_split=0.1,
          shuffle=True)

# Test model
y_pred = model.predict([test_user_data, test_movie_data])
y_true = df_test1['rating'].values

#  Compute RMSE
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result With Keras Matrix-Factorization: {:.4f} RMSE'.format(rmse))

Train on 24138 samples, validate on 2682 samples
Epoch 1/20
24138/24138 [==============================] - 0s 18us/step - loss: 12.0651 - val_loss: 13.1523
Epoch 2/20
24138/24138 [==============================] - 0s 9us/step - loss: 11.9841 - val_loss: 13.1311
Epoch 3/20
24138/24138 [==============================] - 0s 8us/step - loss: 11.4782 - val_loss: 13.0728
Epoch 4/20
24138/24138 [==============================] - 0s 8us/step - loss: 10.2123 - val_loss: 12.9664
Epoch 5/20
24138/24138 [==============================] - 0s 8us/step - loss: 8.3313 - val_loss: 12.8087
Epoch 6/20
24138/24138 [==============================] - 0s 8us/step - loss: 6.3064 - val_loss: 12.6279
Epoch 7/20
24138/24138 [==============================] - 0s 8us/step - loss: 4.5579 - val_loss: 12.4409
Epoch 8/20
24138/24138 [==============================] - 0s 8us/step - loss: 3.2763 - val_loss: 12.2534
Epoch 9/20
24138/24138 [==============================] - 0s 8us/step - loss: 2.4380 - val_loss: 12.0953
E

In [134]:
# Model Architecture
import keras 
from keras.layers import merge

# Create user- & movie-id mapping
user_id_mapping = {id:i for i, id in enumerate(df_ratings['userId'].unique())}
movie_id_mapping = {id:i for i, id in enumerate(df_ratings['movieId'].unique())}


# Create correctly mapped train- & testset
train_user_data = df_train['userId'].map(user_id_mapping)
train_movie_data = df_train['movieId'].map(movie_id_mapping)

test_user_data = df_test1['userId'].map(user_id_mapping)
test_movie_data = df_test1['movieId'].map(movie_id_mapping)


n_latent_factors_user = 8
n_latent_factors_movie = 10
n_latent_factors_mf = 3
n_users, n_movies = len(df_ratings.userId.unique()), len(df_ratings.movieId.unique())

movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding_mlp = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding-MLP')(movie_input)
movie_vec_mlp = keras.layers.Flatten(name='FlattenMovies-MLP')(movie_embedding_mlp)
movie_vec_mlp = keras.layers.Dropout(0.2)(movie_vec_mlp)

movie_embedding_mf = keras.layers.Embedding(n_movies + 1, n_latent_factors_mf, name='Movie-Embedding-MF')(movie_input)
movie_vec_mf = keras.layers.Flatten(name='FlattenMovies-MF')(movie_embedding_mf)
movie_vec_mf = keras.layers.Dropout(0.2)(movie_vec_mf)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec_mlp = keras.layers.Flatten(name='FlattenUsers-MLP')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding-MLP')(user_input))
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_vec_mf = keras.layers.Flatten(name='FlattenUsers-MF')(keras.layers.Embedding(n_users + 1, n_latent_factors_mf,name='User-Embedding-MF')(user_input))
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)


# concat = keras.layers.merge.Concatenate(axis=1)([movie_vec_mlp, user_vec_mlp],name='Concat')
# concat = keras.layers.concatenate([movie_vec_mlp, user_vec_mlp], mode='concat',name='Concat')
concat = keras.layers.concatenate([movie_vec_mlp, user_vec_mlp],name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = keras.layers.BatchNormalization(name='Batch')(dense)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout-1')(dense_batch)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name='Batch-2')(dense_2)


dropout_2 = keras.layers.Dropout(0.2,name='Dropout-2')(dense_batch_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dropout_2)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

pred_mf = keras.layers.Dot(1, normalize=False)([movie_vec_mf, user_vec_mf])


pred_mlp = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)

# combine_mlp_mf = keras.layers.merge([pred_mf, pred_mlp], mode='concat',name='Concat-MF-MLP')
combine_mlp_mf = keras.layers.concatenate([pred_mf, pred_mlp],name='Concat-MF-MLP')
result_combine = keras.layers.Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = keras.layers.Dense(100,name='FullyConnected-4')(result_combine)


result = keras.layers.Dense(1,name='Prediction')(deep_combine)


model_nmf= keras.Model([user_input, movie_input], result)
opt = keras.optimizers.Adam(lr =0.01)
model_nmf.compile(optimizer='adam',loss= 'mean_absolute_error')

In [135]:
history = model_nmf.fit([train_user_data, train_movie_data], df_train.rating, epochs=25, verbose=0, validation_split=0.1)

In [136]:
#  Compute RMSE
y_true = df_test1.rating
y_pred=model_nmf.predict([test_user_data, test_movie_data])
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('\n\nTesting Result with Neural Collaborative MF: {:.4f} RMSE'.format(rmse))



Testing Result with Neural Collaborative MF: 1.0174 RMSE


# Hybrid Modelling using User Profile and NCF

In [137]:
#make recommendations for all members in the test data
# Get the recent 20 movies rated by the user and find 5 most similar movies t
recs = []
for user in df_test.index:
    df_user=df_train[df_train['userId']==user]
    df_user_links = df_links.merge(df_user,on='movieId')
    df_user_links.drop(df_user_links[df_user_links['tmdbId'].isnull()].index,inplace=True)
    df_user_links['id']=df_user_links['tmdbId'].astype(int)
    df_user_links.drop('tmdbId',axis=1,inplace=True)
    df_user_data_with_tags = df_md.reset_index().merge(df_user_links, on='id')
    user_profile = np.dot(tfidf_matrix_soup[idx2movie[df_user_data_with_tags['index'].values].values].toarray().T, df_user_data_with_tags['rating'].values)
    C = cosine_similarity(np.atleast_2d(user_profile), tfidf_matrix_soup)
    R = np.argsort(C)[:, ::-1]
    recommendations = [i for i in R[0] if i not in df_user_data_with_tags['index'].values]
#     recs.append(df_smd['index'][recommendations].head(130).values.tolist())
    
# Test model NCF
    df_test_movie = df_ratings[['movieId']].drop_duplicates()
    test_movie_data1 = df_test_movie['movieId'].map(movie_id_mapping)
    test_user_data1 = np.array([user_id_mapping[user] for i in range(len(df_test_movie))])
    y_pred = model.predict([test_user_data1,test_movie_data1]) 
    df_test_movie['Predicted_Rating'] = y_pred
    df_test_movie=df_test_movie.sort_values('Predicted_Rating',ascending=False)
    
    recs.append(df_test_movie['movieId'].head(75).values.tolist()
               + df_smd['index'][recommendations].head(75).values.tolist())
    
df_test['cf_predictions_hybrid'] = recs
df_test.head()

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a,cf_predictions_kcdg,l_kcdg,cf_predictions_kcdg2,cf_predictions_up,l_up,cf_predictions_hybrid
userId,,,,,,,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340,"[523, 3296, 988, 4119, 5538, 100, 6592, 8394, ...",150,"[1918, 3237, 271, 7306, 8370, 919, 4827, 3427,...","[5359, 5932, 5927, 5934, 5928, 5929, 5930, 593...",150,"[246, 296, 858, 994, 1267, 608, 912, 111, 3022..."
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146,"[5133, 1539, 4977, 2363, 5075, 1584, 4146, 604...",150,"[2122, 1822, 329, 2363, 4977, 1192, 2207, 7228...","[7416, 7414, 7419, 7415, 7420, 7418, 7421, 741...",150,"[246, 296, 858, 1267, 608, 111, 994, 912, 4226..."
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203,"[535, 6881, 6948, 2557, 1853, 8666, 8226, 1463...",150,"[2899, 5167, 8379, 3245, 2106, 3433, 1463, 121...","[5862, 4505, 16076, 8911, 1220, 2321, 1614, 32...",150,"[246, 296, 994, 858, 608, 1267, 912, 111, 1147..."
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103,"[1056, 4619, 6476, 7632, 9100, 4951, 4960, 886...",150,"[2058, 7421, 5496, 8811, 9100, 5255, 7650, 239...","[5529, 6281, 2911, 7016, 9786, 25543, 5895, 13...",150,"[246, 296, 994, 858, 608, 1267, 912, 3730, 114..."
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105,"[1077, 8180, 7061, 3235, 8612, 8456, 8435, 766...",150,"[8145, 8474, 306, 7061, 1789, 4796, 2978, 5093...","[5933, 5930, 5928, 5932, 5929, 5934, 5931, 592...",150,"[246, 296, 994, 858, 608, 912, 1267, 1147, 373..."


In [138]:
df_test['l_hybrid']=df_test['cf_predictions_hybrid'].apply(lambda x: len(x))
df_test.head()

,actual,cf_predictions,Length,cf_predictions_td,L_td,l_a,cf_predictions_kcdg,l_kcdg,cf_predictions_kcdg2,cf_predictions_up,l_up,cf_predictions_hybrid,l_hybrid
userId,,,,,,,,,,,,,
15,"[54272, 115713, 122882, 114180, 7173, 111622, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",78,"[5669, 4136, 686, 6705, 5926, 2087, 246, 1430,...",150,340,"[523, 3296, 988, 4119, 5538, 100, 6592, 8394, ...",150,"[1918, 3237, 271, 7306, 8370, 919, 4827, 3427,...","[5359, 5932, 5927, 5934, 5928, 5929, 5930, 593...",150,"[246, 296, 858, 994, 1267, 608, 912, 111, 3022...",150
23,"[1, 33794, 6, 11, 2571, 1556, 42004, 535, 32, ...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",195,"[3050, 5795, 5344, 1822, 5815, 6560, 236, 8187...",150,146,"[5133, 1539, 4977, 2363, 5075, 1584, 4146, 604...",150,"[2122, 1822, 329, 2363, 4977, 1192, 2207, 7228...","[7416, 7414, 7419, 7415, 7420, 7418, 7421, 741...",150,"[246, 296, 858, 1267, 608, 111, 994, 912, 4226...",150
30,"[5120, 5122, 5125, 5127, 3082, 5655, 4122, 515...","[10309, 43320, 12481, 22879, 2843, 314, 351, 1...",196,"[7118, 7231, 6060, 8001, 7232, 3197, 2779, 850...",150,203,"[535, 6881, 6948, 2557, 1853, 8666, 8226, 1463...",150,"[2899, 5167, 8379, 3245, 2106, 3433, 1463, 121...","[5862, 4505, 16076, 8911, 1220, 2321, 1614, 32...",150,"[246, 296, 994, 858, 608, 1267, 912, 111, 1147...",150
48,"[114180, 109578, 27660, 106002, 34323, 27156, ...","[10309, 12481, 22879, 2843, 314, 351, 834, 248...",89,"[1386, 5325, 8155, 2500, 8684, 6539, 141, 3381...",150,103,"[1056, 4619, 6476, 7632, 9100, 4951, 4960, 886...",150,"[2058, 7421, 5496, 8811, 9100, 5255, 7650, 239...","[5529, 6281, 2911, 7016, 9786, 25543, 5895, 13...",150,"[246, 296, 994, 858, 608, 1267, 912, 3730, 114...",150
56,"[2052, 4102, 3079, 122886, 2572, 55820, 152081...","[10309, 43320, 12481, 22879, 2843, 314, 351, 8...",171,"[3021, 8001, 8913, 4860, 8557, 3608, 4190, 671...",150,105,"[1077, 8180, 7061, 3235, 8612, 8456, 8435, 766...",150,"[8145, 8474, 306, 7061, 1789, 4796, 2978, 5093...","[5933, 5930, 5928, 5932, 5929, 5934, 5931, 592...",150,"[246, 296, 994, 858, 608, 912, 1267, 1147, 373...",150


In [139]:
# At K=25
actual = df_test.actual.values.tolist()
cf_predictions_hybrid = df_test.cf_predictions_hybrid.values.tolist()
print('MAR@K25:',mark(actual, cf_predictions_hybrid, k=25))
print('MAR@K50:',mark(actual, cf_predictions_hybrid, k=50))
print('MAR@K75:',mark(actual, cf_predictions_hybrid, k=75))
print('MAR@K100:',mark(actual, cf_predictions_hybrid, k=100))
print('MAR@K125:',mark(actual, cf_predictions_hybrid, k=125))
print('MAR@K150:',mark(actual, cf_predictions_hybrid, k=150))

MAR@K25: 0.14277181959895727
MAR@K50: 0.1952377363571024
MAR@K75: 0.2689526489487886
MAR@K100: 0.2853129102308775
MAR@K125: 0.3011163850950049
MAR@K150: 0.3093317386551015


In [140]:
print('Precision for RS based on User Profile is:',recommender_precision(actual,cf_predictions_hybrid))

Precision for RS based on User Profile is: 2.988718918918919
